# Argumenty a funkce

In [ ]:
# Vynucení kontroly souladu s PEP8
!pip install flake8 pycodestyle pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on

### 1. Napište funkci, která připojí všechna kladná čísla z jednoho seznamu na konec druhého předaného seznamu a vrátí upravený seznam.

Operaci provádějte _inplace_, tj. upravujte druhý předaný seznam.

Druhý seznam bude **volitelný parametr**, který bude mít jako výchozí hodnotu *prázdný seznam* (pokud není zadán).

__Hint__: Pozor na výchozí argumenty, které jsou _mutable_.

In [ ]:
# chybné řešení
# def append_positive_inplace(numbers, target=[]):
#     [target.append(num) for num in numbers if num > 0]
#     return target

def append_positive_inplace(numbers, target=None):
    if target is None:
        target = []
    [target.append(num) for num in numbers if num > 0]
    return target

In [ ]:
print(append_positive_inplace([-1, 1, 2, -3]))  # [1, 2]
print(append_positive_inplace([1, 5, -5], [10, 20]))  # [10, 20, 1, 5]
print(append_positive_inplace([-2, 2, 4]))  # [2, 4]

### 2. Napište funkci, která sčítá čísla na stejných pozicích v předaných seznamech.

Funkce bude příjmat jako argumenty libovolný počet číselných seznamů a vracet seznam součtů. Pokud mají seznamy různou délku, doplňte všechny kratší seznamy nulami.

**Hint**: `zip()`

In [ ]:
def sum_iterables(*lists):
    max_len = max([len(ls) for ls in lists])
    lists_padded = [ls + [0] * (max_len - len(ls)) for ls in lists]
    return [sum(x) for x in zip(*lists_padded)]

In [ ]:
print(sum_iterables([1, 2, 3], [4, 5, 6]))  # [5, 7, 9]
print(sum_iterables([1, 2, 3], [4, 5, 6], [1, 1, 1]))  # [6, 8, 10]
print(sum_iterables([1, 2, 3], [10], [0, 5], []))  # [11, 7, 3]

### 3. Zobecněte příklad 2. aby bylo možno operaci nad prvky seznamu předat parametrem jako funkci

Zároveň zaveďte volitelný parametr pro _padding_, kterým se budou doplňovat kratší seznamy.

**Hint**: Většina kódu by měla zůstat stejná. Pouze parametrizujte existující funkci.

In [ ]:
def merge_iterables(*lists, op, padding=[None]):
    max_len = max([len(ls) for ls in lists])
    lists_padded = [ls + padding * (max_len - len(ls)) for ls in lists]
    return [op(x) for x in zip(*lists_padded)]

In [ ]:
print(
    merge_iterables(
        [1, 2, 3],
        [4, 5, 6],
        [1, 1, 1],
        op=sum,
    ),
)  # [6, 8, 10])

print(
    merge_iterables(
        [1, 2, 3],
        [10],
        [0, 5],
        [],
        op=sum,
        padding=[0],
    )
)  # [11, 7, 3]

print(
    merge_iterables(
        "FBBX",
        "oaao",
        "orz",
        op=lambda x: ''.join(x), padding="_",
    )
)  # ['Foo', 'Bar', 'Baz', 'Xo_']

### 4. Napište funkci, která vrací složení funkcí.

Funkce bude brát libovolný počet funkcí jako argumenty a bude vracet funkci, která bude jejich složením v pořadí, ve kterém jsou uvedeny.

Příklad: `compose(f, g, h) -> h(g(f(x)))`

In [ ]:
from functools import reduce


def compose(*fns):
    def composed(x):
        for fn in fns:
            x = fn(x)
        return x
    return composed


# functional solution
def compose_functional(*fns):
    return lambda x: reduce(lambda acc, fn: fn(acc), fns, x)

In [ ]:
import math
f = compose(lambda x: x**2, lambda x: x//2, math.sqrt)
print(f(1))
print(f(2))
print(f(5))

### 5. Napište funkci, která bude provádět (sekvenční) map-reduce nad seznamem.

Funkce dostane na vstupu seznam, mapovací funkci a redukční funkci.

Mapovací funkce se aplikuje na všechny prvky seznamu a její výsledky se poté ve dvojicích předávájí redukční funkci, dokud nezbyde pouze jedna hodnota - výsledek redukce.

Argument pro mapovací funkci bude mít jako výchozí hodnotu identitu (tj. vrací vstup).

Vynuťte, aby argumenty pro mapovací a redukční funkci bylo nutno zadat přes pojmenované argumenty (tj. aby je nešlo předat přes poziční argument).

In [ ]:
def list_map_reduce(input_list, *, map_fn=lambda x: x, reduce_fn):
    mapped = [map_fn(x) for x in input_list]
    acc = mapped[0]
    for x in mapped[1:]:
        acc = reduce_fn(acc, x)
    return acc

In [ ]:
# součet pole - 6
print(
    list_map_reduce([1, 2, 3], reduce_fn=lambda x, y: x + y)
)

# suma čtverců - 14
print(
    list_map_reduce(
        [1, 2, 3],
        map_fn=lambda x: x**2, reduce_fn=lambda x, y: x + y,
    ),
)

# délka nejdelšího slova v seznamu - 5
print(
    list_map_reduce(
        ["Hello", "yes", "this", "is", "dog"],
        map_fn=len,
        reduce_fn=lambda x, y: max([x, y]),
    )
)

# transformace na kapitálky a spojení seznamu slov - ARE YOU KIDDING ME ?!!
print(
    list_map_reduce(
        ["Are", "you", "kidding", "me", "?!"],
        map_fn=lambda x: x.upper(),
        reduce_fn=lambda x, y: '  '.join([x, y]),
    )
)

### 6. Napište funkci, která umožní zafixovat argumenty libovolné předané funkce

Funkce příjmá funkci pro obalení a libovolné další poziční a pojmenované argumenty a vrací funkci, která se chová stejně jako předaná funkce s výchozí hodnotou pro předané argumenty.

Zafixované funkci je možné předat libovolné další argumenty při jejím volání. Poziční argumenty se budou spojovat za sebe (první jdou argumenty předané obalené funkci a poté zafixované) a pojmenované argumenty se přepisují (prioritu má argument při volání obalené funkce).

_Zkuste ve vaší funkci využít lambda funkci._

**Hint**: Budete potřebovat spojení dvou seznamů a slovníků.

In [ ]:
def wrap(fn, *args, **kwargs):
    return lambda *args2, **kwargs2: fn(*(args2+args), **{**kwargs, **kwargs2})

In [ ]:
import sys
shout = wrap(print, "!!!!", sep='', flush=True)
shout("This is madness")  # This is madness!!!!

# THIS  IS  SPARTA !!!! (in red)
shout("THIS", "IS", "SPARTA", sep='  ', file=sys.stderr)

fixed_end_range = wrap(range, 10)
print(list(fixed_end_range(5)))  # [5, 6, 7, 8, 9]
print(list(fixed_end_range(0)))  # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]